### Parses all champ stats from a hosted server
This allows us to retrieve descriptions for all abilities and retrieve ability related info.

In [2]:
import os

CHAMP_STATS_URL = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/champions/"

In [3]:
import requests
from bs4 import BeautifulSoup

page = requests.get(CHAMP_STATS_URL)
soup = BeautifulSoup(page.content, "html.parser")
champ_hrefs = soup.find_all(href=lambda x: "json" in x if x else False)
champ_jsons = [champ_href.contents[0] for champ_href in champ_hrefs]
champ_json_urls = [f"{CHAMP_STATS_URL}{champ_json}" for champ_json in champ_jsons]

In [4]:
import urllib.request, json

character_abilities = {}
for champ_json_url in champ_json_urls:
    with urllib.request.urlopen(champ_json_url) as url:
        data = json.load(url)
        name = data['name']
        abilities = data['abilities']
        character_abilities[name] = abilities

In [5]:
with open("characters.json", "w+") as f:
    json.dump(character_abilities, f)

### Video scraper scripts
Want to download all the videos that are in the LoL wiki.

In [79]:
from bs4 import BeautifulSoup
import requests

LOL_WIKI = "https://leagueoflegends.fandom.com"
LIST_OF_CHAMPS = LOL_WIKI + "/wiki/List_of_champions"

champs_soup = BeautifulSoup(requests.get(url=LIST_OF_CHAMPS).content, 'html.parser')

In [81]:
nested_links = champs_soup.select('span > span > a')
champ_links = list(set([link['href'] for link in nested_links]))
champ_links = [f"{LOL_WIKI}{link}" for link in champ_links]
champ_links

187

In [82]:
import glob
import re

SAVE_DIR = "Data/Videos"
os.makedirs(SAVE_DIR, exist_ok=True)

for url in champ_links:
    champ_name = url.split("/")[-2]
    if glob.glob(f"{SAVE_DIR}/{champ_name}*"):
        print(f"Already downloaded videos for {champ_name}")
        continue
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    video_links = soup.select('video')
    downloads = [link['src'] for link in video_links]
    if len(downloads) == 0:
        print(f"No links found for: {champ_name}")
        continue
    
    filename_pattern = re.compile(r'/([^/]+\.(webm|ogv))')
    for download in downloads:
        match = filename_pattern.search(download)
        if not match:
            print(f"Failed to extract filename for URL: {download}")
            continue
       
        file_name = match.group(1).replace('Video', '')
        file_path = os.path.join(SAVE_DIR, file_name)
        print(f"Downloading: {download}")
        download_req = requests.get(download, stream=True)
        if download_req.status_code != 200:
            print(f"Failed to download: {download} (Status code: {download_req.status_code})")
            continue
        with open(file_path, 'wb') as file:
            for chunk in download_req.iter_content(chunk_size=1024):
                if chunk:  # Filter out keep-alive chunks
                    file.write(chunk)
        print(f"Saved to {file_path}")                 

No links found for: Senna
No links found for: Galio
No links found for: Zilean
No links found for: Hwei
No links found for: Gragas
Already downloaded videos for Ezreal
No links found for: Janna
No links found for: wiki
No links found for: Briar
No links found for: Rumble
No links found for: wiki
No links found for: Xerath
No links found for: Aurora
Already downloaded videos for Akshan
No links found for: Nilah
No links found for: Tristana
No links found for: Naafiri
No links found for: Vel%27Koz
Already downloaded videos for Ahri
No links found for: wiki
Already downloaded videos for Braum
Already downloaded videos for Cho%27Gath
No links found for: Yone
No links found for: Shyvana
Already downloaded videos for Aphelios
No links found for: Jarvan_IV
No links found for: Maokai
No links found for: Jinx
No links found for: Kayn
No links found for: Jayce
No links found for: Katarina
No links found for: Nami
No links found for: wiki
Already downloaded videos for Fiora
No links found for: Re

KeyboardInterrupt: 